#インストール

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==2.4.0
#!pip install iterative-stratification
!pip install emoji
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 53.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-many

#pytorch定義

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, BertModel
import pytorch_lightning as pl

In [ ]:
"""
pytorch　bertマルチ分類モデルの利用
"""

class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModelのロード
        self.bert = BertModel.from_pretrained(model_name, output_hidden_states=True)
        # 線形変換を初期化しておく
        self.linear = torch.nn.Linear(
            self.bert.config.hidden_size, num_labels
        )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        # データを入力しBERTの最終層の出力を得る。
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        hidden_states = bert_output.hidden_states
        last_hidden_state = bert_output.last_hidden_state

        # [PAD]以外のトークンで隠れ状態の平均をとる
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)

        # 線形変換
        scores = self.linear(averaged_hidden_state)

        # 出力の形式を整える。
        output = {'logits': scores,'hidden_states':hidden_states, 'last_hidden_state': last_hidden_state, 'averaged_hidden_state': averaged_hidden_state}

        #ラベルごとの重み
        """
        データ数 : 766
        {1: 660, 2: 2, 3: 88, 4: 154, 5: 231, 6: 32, 7: 20, 8: 19, 9: 18}
        """
        l_1_wegiht = 1 - 660/766 #766/660 * 0.4 #
        l_2_wegiht = 1 - 88/766 #766/88 * 0.4 #
        l_3_wegiht = 1 - 154/766 #766/154 * 0.4 #
        l_4_wegiht = 1 - 231/766 #766/231 * 0.4 #
        l_5_wegiht = 1 - 32/766 #766/32 * 0.4 #
        l_6_wegiht = 1 - 20/766 #766/20 * 0.4 #
        l_7_wegiht = 1 - 19/766 #766/19 * 0.4 #
        l_8_wegiht = 1 - 18/766 #766/18 * 0.4 #
        pos_wegiht = torch.tensor([l_1_wegiht,l_2_wegiht, l_3_wegiht, l_4_wegiht, l_5_wegiht,l_6_wegiht, l_7_wegiht,l_8_wegiht],device=torch.device('cuda:0'))

        # labelsが入力に含まれていたら、損失を計算し出力する。
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss(pos_weight=pos_wegiht)(scores, labels.float())
            output['loss'] = loss

        # 属性でアクセスできるようにする。
        output = type('bert_output', (object,), output)

        return output

In [ ]:
"""
pl定義
"""
#test_labels = []
#test_labels_predicted = []

class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        self.save_hyperparameters()

        # BERTのロード
        self.bert_scml = BertForSequenceClassificationMultiLabel(
          model_name, num_labels = num_labels
        )
        self.test_labels = []
        self.test_labels_predicted = []

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    def test_step(self, batch, batch_idx):
        #global test_labels, test_labels_predicted
        labels = batch.pop('labels') # バッチからラベルを取得
        self.log('labels', labels)

        self.test_labels = labels.to('cpu').detach().numpy().copy() #numpy配列変換
        output = self.bert_scml(**batch)
        scores = output.logits
        #self.log('scores', scores)
        #print(scores)
        labels_predicted = (scores > 0).int()

        self.test_labels_predicted = labels_predicted.to('cpu').detach().numpy().copy()
        #self.log('labels_predicted', labels_predicted)
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0) #
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

#データセット用意

In [ ]:
import pandas as pd
import emoji

#レビューリストとラベルリストを返す関数
def csv_to_list(csv_list : list):
  review_list,label_list = [],[]
  label_num = 8 #分類カテゴリ数
  each_label_num = {} #各label数のカウント
  #each_label_numのvalueを初期化
  for i in range(label_num):
    each_label_num[i+1] = 0
  for i in range(len(csv_list)):
    review = emoji.replace_emoji(csv_list[i][1])
    review_list.append(review)
    #csv_list[i][2]がintではなかったら
    if not isinstance(csv_list[i][2], int):
      str_category_list = csv_list[i][2].split(',')
    else:
      str_category_list = [csv_list[i][2]]
    try:
      category_list = [int(num) for num in str_category_list]
    except ValueError:
      print(i)
    onehot_lavel = []
    #空のマルチラベルリスト作成
    for i in range(label_num):
      onehot_lavel.append(0)
    for category in category_list:
      onehot_lavel[int(category)-1] = 1
      each_label_num[int(category)] =  each_label_num[int(category)] + 1
    label_list.append(onehot_lavel)
  return review_list, label_list, each_label_num

def multi_label_to_single_label(multi_label_list, target_label):
  single_label_list = []
  for i, _label in enumerate(multi_label_list):
    single_label_list.append(multi_label_list[i][target_label - 1])
  return single_label_list

#[0,1,1,0...0]の様なリストからtarget(0 or 1)であるリストのインデックスを返す関数
def get_label_index(label_list, target_label):
  label_index_list = []
  for i, _label in enumerate(label_list):
    if _label == target_label:
      label_index_list.append(i)
  return label_index_list

#インデックスリストを元にreviewsとlabelsの両方のリストから抽出したリストを返す関数
def get_review_label_from_index(label_index_list, reviews, labels):
  return [reviews[i] for i in label_index_list], [labels[i] for i in label_index_list]


# CSVファイルの読み込み
data = pd.read_csv('drive/MyDrive/review_ver0510.csv')
review_label_list = data.values.tolist()
#オーグメンテーション用
data = pd.read_csv('drive/MyDrive/aug_review_ver0510.csv')
aug_review_label_list = data.values.tolist()
#シングルラベル用
data = pd.read_csv('drive/MyDrive/single_label_review.csv')
single_label_review_label_list = data.values.tolist()

review_list,label_list, each_label_num = csv_to_list(review_label_list)
aug_review_list, aug_label_list, aug_each_label_num = csv_to_list(aug_review_label_list)
single_label_review_list, single_label_label_list, single_label_each_label_num = csv_to_list(single_label_review_label_list)

In [ ]:
import os
model_path = '/content/drive/My Drive/path_to_save_model/'
model_path = model_path + os.listdir(model_path)[1]
print(model_path)

/content/drive/My Drive/path_to_save_model/epoch=49-step=1050-v4.ckpt


#モデルロード，推論動作確認

In [ ]:
"""
推論モードモデルの動作確認
"""
import os

MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
# Google Driveに保存したモデルのパス
model_path = '/content/drive/My Drive/path_to_save_model/'
model_path = model_path + os.listdir(model_path)[1]
print(model_path)
# トークナイザーをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# モデルをロード
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(model_path)
bert_scml = model.bert_scml
# モデルの評価モードを設定
model.eval()
# テストデータのトークナイズ
texts = ["2万円を切るお手ごろ価格。たまに訪れる別宅で、在室時だけ電源を入れる、という使い方には十分の性能。継続してつかいつづけるには、さすがに小さくてパワー不足だけど。", "寝室に置いてますが、音も静かで、。アイスやドリンクを入れています。がじゅうぶんな大きさです。"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

# モデルの予測を取得
with torch.no_grad():
    outputs = bert_scml(**inputs)
    predictions = outputs.logits
labels_predicted = (predictions > 0).int().cpu().numpy().tolist()
print(labels_predicted)
# 予測結果の確認
print(predictions)

/content/drive/My Drive/path_to_save_model/epoch=49-step=1050-v4.ckpt


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0]]
tensor([[ 1.7511, -2.5439, -0.1660,  1.9490, -4.6795, -3.3460, -4.5604, -4.0528],
        [ 1.3965, -3.8467, -3.0943,  1.7587, -4.1931, -3.8143, -3.2974, -3.4910]])


# データ整形

In [ ]:
from sklearn.model_selection import train_test_split
import random
labels = label_list + aug_label_list
labels = multi_label_to_single_label(labels, 2)#ラベル2の2値分類のためsingleラベル変換
reviews = review_list + aug_review_list
positive_taget_index = get_label_index(labels, 1)
positive_reviews, positive_labels = get_review_label_from_index(positive_taget_index, reviews, labels)
"""
#ラベルの共起を回避するために作成したラベル2用のコード
#positive_reviewsをsingle_label_review利用
positive_reviews = single_label_review_list
positive_labels = multi_label_to_single_label(single_label_label_list, 2)
"""

negative_taget_index = get_label_index(labels, 0)
negative_reviews, negative_labels = get_review_label_from_index(negative_taget_index, reviews, labels)
#negativeの方がpositiveより多いので数を揃える

negative_reviews = negative_reviews[:len(positive_reviews)]
negative_labels = negative_labels[:len(positive_reviews)]

print(len(negative_reviews))
reviews = positive_reviews + negative_reviews
labels = positive_labels + negative_labels
"""
combined = list(zip(reviews, labels))
# 結合したリストをシャッフル
random.shuffle(combined)
# 再分割
shuffled_review_list, shuffled_label_list = zip(*combined)
# リストに戻す
reviews = list(shuffled_review_list)
labels = list(shuffled_label_list)
#カット
reviews = reviews[:400]
labels = labels[:400]
"""
# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    reviews, labels, test_size=0.5, stratify=labels
)
print(len(reviews))

830
964


# 隠れ層状態確認

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold

def get_hidden_states(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = bert_scml(**inputs)
    hidden_states = outputs.hidden_states
    return hidden_states

train_hidden_states = get_hidden_states(train_texts, model, tokenizer)
test_hidden_states = get_hidden_states(test_texts, model, tokenizer)

print(f"1st: {len(train_hidden_states)=}") #tuple, (layers) = 13
print(f"1st: {len(train_hidden_states[0])=}") #tuple, (layers, samples) = 13, 200
print(f"1st: {len(train_hidden_states[0][0])=}") #tuple, (layers, samples, sequences) = 13, 200, 21
print(f"1st: {len(train_hidden_states[0][0][0])=}") #tuple, (layers, samples, sequences, embeddings) = 13, 200, 21, 768

1st: len(train_hidden_states)=13
1st: len(train_hidden_states[0])=482
1st: len(train_hidden_states[0][0])=128
1st: len(train_hidden_states[0][0][0])=768


In [ ]:
# Flatten hidden states (シーケンス長異なると困るので、トークン単位での平均を求める)
# hは (samples, sequences, embeddings)。
# dim=1 で (dim=0, dim=1, dim=2) のインデックス指定になり、sequencesの平均を求めている
train_hidden_states = [torch.mean(h, dim=1).numpy() for h in train_hidden_states]
test_hidden_states = [torch.mean(h, dim=1).numpy() for h in test_hidden_states]
print(f"flatten: {len(train_hidden_states)=}") #list (layers) = 13
print(f"flatten: {len(train_hidden_states[0])=}") #list (layers, samples) = 13, 200
print(f"flatten: {len(train_hidden_states[0][0])=}") #list (layers, samples, embeddings) = 13, 200, 768

flatten: len(train_hidden_states)=13
flatten: len(train_hidden_states[0])=482
flatten: len(train_hidden_states[0][0])=768


In [ ]:
# reshapeするためnp.arrayに変換 (layers, samples, embeddings)
train_hidden_states = np.stack(train_hidden_states, axis=0)
test_hidden_states = np.stack(test_hidden_states, axis=0)
print(f"stacked: {train_hidden_states.shape=}")

stacked: train_hidden_states.shape=(13, 482, 768)


In [ ]:
# Reshape to ensure the correct dimensions
#train_hidden_states = np.array([h for h in train_hidden_states])
#test_hidden_states = np.array([h for h in test_hidden_states])
#print(f"reshaped: {train_hidden_states.shape=}")

# train_hidden_statesの要素順を入れ替え。
# np.transpose()の(1,0,2)は、
#    元の1次元目(samples)を0次元目に移動。
#    元の0次元目(layers)を1次元目に移動。
#    元の2次元目(embeddings)を2次元目にそのまま保持。
# 元の順序: (layers=0, samples=1, embeddings=2)
#   新順序: (samples=0, layers=1, embeddings=2)
train_hidden_states = np.transpose(train_hidden_states, (1, 0, 2))
test_hidden_states = np.transpose(test_hidden_states, (1, 0, 2))
print(f"transposed: {train_hidden_states.shape=}")

transposed: train_hidden_states.shape=(482, 13, 768)


#プロービング

In [ ]:
#train_hidden_states内容確認
print(train_hidden_states[1, 3, : 100])

[ 0.0328475  -0.29284418  0.13327196 -0.32360077 -0.09735079  0.22174872
 -0.01717078  0.0328992  -0.5107652  -0.1918315   0.06258428  0.16113308
 -0.02431575 -0.05036568  0.04060321 -0.3487125   0.15391047  0.03003525
  0.26707733  0.44848645  0.11799397 -0.05358445  0.15283096  0.4301954
 -0.14450283  0.28029382 -0.13915938 -0.20503667  0.00556966  0.38209555
 -0.02737822  0.17576446  0.2037715   0.08453561  0.2344648  -0.10059606
 -0.33118656  0.03579221  0.10922978 -0.07289358 -0.10476018 -0.00685234
  0.12910332  0.36681163  0.05044168 -0.3675759  -0.23114824  0.2955515
 -0.08893213  0.0723777   0.05516198  0.1687974   0.17736794  0.50784385
 -0.17398706 -0.15237302 -0.05039914  0.1649677   0.09734482 -0.37545615
  0.15502703 -0.00712691 -0.4791668  -0.2069171  -0.07629459  0.08919368
 -0.28341034 -0.5267576  -0.23588726 -0.1097003  -0.05418605  0.2502601
 -0.10446417 -0.35020614 -0.22321984 -0.01129734 -0.15550908  0.11526852
  0.47919184  0.2729613   0.13885237 -0.0610039  -0.01

In [ ]:
# Step 3: プロービング
accuracies = []

for layer in range(train_hidden_states.shape[1]):
    X_train = train_hidden_states[:, layer, :]
    #print(f"{X_train.shape=}")
    X_test = test_hidden_states[:, layer, :]

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, train_labels)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(test_labels, predictions)
    accuracies.append(accuracy)
    #print(f"Layer {layer + 1}: Test Accuracy = {accuracy:.4f}")

# 最終的な分類精度の表示
for layer, accuracy in enumerate(accuracies, 1):
    print(f"Layer {layer}: Test Accuracy = {accuracy:.4f}")

Layer 1: Test Accuracy = 0.8672
Layer 2: Test Accuracy = 0.9046
Layer 3: Test Accuracy = 0.9087
Layer 4: Test Accuracy = 0.9129
Layer 5: Test Accuracy = 0.9274
Layer 6: Test Accuracy = 0.9378
Layer 7: Test Accuracy = 0.9357
Layer 8: Test Accuracy = 0.9357
Layer 9: Test Accuracy = 0.9357
Layer 10: Test Accuracy = 0.9357
Layer 11: Test Accuracy = 0.9336
Layer 12: Test Accuracy = 0.9398
Layer 13: Test Accuracy = 0.9440


# 制御タスク

In [ ]:
# シャッフルしたラベルを用意
import random
control_labels = [1] * len(positive_labels) + [0] * len(negative_labels)
random.shuffle(control_labels)
control_labels[:10]

[0, 0, 1, 0, 0, 1, 1, 0, 1, 0]

In [ ]:
print(len(control_labels))

964


In [ ]:
control_train_labels = control_labels[:len(train_labels)]
control_test_labels = control_labels[len(train_labels):]

In [ ]:
# Step 3: プロービング
accuracies = []

for layer in range(train_hidden_states.shape[1]):
    X_train = train_hidden_states[:, layer, :]
    #print(f"{X_train.shape=}")
    X_test = test_hidden_states[:, layer, :]

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, control_train_labels)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(control_test_labels, predictions)
    accuracies.append(accuracy)
    #print(f"Layer {layer + 1}: Control Test Accuracy = {accuracy:.4f}")

# 最終的な分類精度の表示
for layer, accuracy in enumerate(accuracies, 1):
    print(f"Layer {layer}: Control Test Accuracy = {accuracy:.4f}")

Layer 1: Control Test Accuracy = 0.8651
Layer 2: Control Test Accuracy = 0.8548
Layer 3: Control Test Accuracy = 0.8465
Layer 4: Control Test Accuracy = 0.8485
Layer 5: Control Test Accuracy = 0.8568
Layer 6: Control Test Accuracy = 0.8610
Layer 7: Control Test Accuracy = 0.8527
Layer 8: Control Test Accuracy = 0.8568
Layer 9: Control Test Accuracy = 0.8402
Layer 10: Control Test Accuracy = 0.8320
Layer 11: Control Test Accuracy = 0.8154
Layer 12: Control Test Accuracy = 0.8008
Layer 13: Control Test Accuracy = 0.7905
